In [26]:
# Load and display the entire log file (use with caution for large files)
log_file_path = "C:/Users/AU/Documents/Nagesh N/Log Analysis Script/sample.log"

with open(log_file_path, 'r') as file:
    log_lines = file.readlines()

# Display all lines
for line in log_lines:
    print(line.strip())

# Optional: Display line count for reference
print(f"\nTotal lines in log file: {len(log_lines)}")

192.168.1.1 - - [03/Dec/2024:10:12:34 +0000] "GET /home HTTP/1.1" 200 512
203.0.113.5 - - [03/Dec/2024:10:12:35 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
10.0.0.2 - - [03/Dec/2024:10:12:36 +0000] "GET /about HTTP/1.1" 200 256
192.168.1.1 - - [03/Dec/2024:10:12:37 +0000] "GET /contact HTTP/1.1" 200 312
198.51.100.23 - - [03/Dec/2024:10:12:38 +0000] "POST /register HTTP/1.1" 200 128
203.0.113.5 - - [03/Dec/2024:10:12:39 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
192.168.1.100 - - [03/Dec/2024:10:12:40 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
10.0.0.2 - - [03/Dec/2024:10:12:41 +0000] "GET /dashboard HTTP/1.1" 200 1024
198.51.100.23 - - [03/Dec/2024:10:12:42 +0000] "GET /about HTTP/1.1" 200 256
192.168.1.1 - - [03/Dec/2024:10:12:43 +0000] "GET /dashboard HTTP/1.1" 200 1024
203.0.113.5 - - [03/Dec/2024:10:12:44 +0000] "POST /login HTTP/1.1" 401 128 "Invalid credentials"
203.0.113.5 - - [03/Dec/2024:10:12:45 +0000] "POST /login HTTP/1.1" 

In [27]:
from collections import Counter

# Extract and count requests per IP address
def count_requests_per_ip(log_lines):
    ip_counter = Counter()
    
    for line in log_lines:
        # Assuming the IP address is the first word in each log entry
        ip = line.split()[0]
        ip_counter[ip] += 1

    # Sort by request count in descending order
    sorted_ip_counts = ip_counter.most_common()
    return sorted_ip_counts

# Count and display the results
ip_request_counts = count_requests_per_ip(log_lines)

# Print results in a formatted table
print("Count Request Per IP Address")
print(f"\n{'IP Address':<20}{'Request Count':<15}")
print("-" * 35)
for ip, count in ip_request_counts:
    print(f"{ip:<20}{count:<15}")


Count Request Per IP Address

IP Address          Request Count  
-----------------------------------
203.0.113.5         8              
198.51.100.23       8              
192.168.1.1         7              
10.0.0.2            6              
192.168.1.100       5              


In [28]:
from collections import Counter

# Extract all unique endpoints and their access counts
def extract_all_endpoints_and_most_accessed(log_lines):
    endpoint_counter = Counter()

    for line in log_lines:
        try:
            # Assuming the endpoint (URL or path) is the 7th element in the log line
            parts = line.split()
            endpoint = parts[6]  # Adjust index if log format differs
            endpoint_counter[endpoint] += 1
        except IndexError:
            continue  # Skip lines that don't match the expected format

    # Find the most accessed endpoint
    most_accessed = endpoint_counter.most_common(1)

    return endpoint_counter, most_accessed

# Extract and analyze endpoints
endpoints_counter, most_accessed_endpoint = extract_all_endpoints_and_most_accessed(log_lines)

# Display all unique endpoints
print("\nExtracted Endpoints and Their Access Counts:")
print(f"\n{'Endpoint':<50}{'Access Count':<15}")
print("-" * 65)
for endpoint, count in endpoints_counter.items():
    print(f"{endpoint:<50}{count:<15}")

# Display the most accessed endpoint
print("\nMost frequently Accessed Endpoint:")
if most_accessed_endpoint:
    endpoint, count = most_accessed_endpoint[0]
    print(f"\n{'Endpoint':<50}{'Access Count':<15}")
    print("-" * 65)
    print(f"{endpoint:<50}{count:<15}")
else:
    print("No endpoints found in the log.")



Extracted Endpoints and Their Access Counts:

Endpoint                                          Access Count   
-----------------------------------------------------------------
/home                                             5              
/login                                            13             
/about                                            5              
/contact                                          2              
/register                                         2              
/dashboard                                        3              
/profile                                          2              
/feedback                                         2              

Most frequently Accessed Endpoint:

Endpoint                                          Access Count   
-----------------------------------------------------------------
/login                                            13             


In [29]:
# Detect suspicious activity (failed login attempts)
# Flagging IP addresses with failed login attempts exceeding a configurable threshold (default: 10 attempts).
def detect_suspicious_activity(log_lines, threshold=10):
    suspicious_ips = Counter()

    for line in log_lines:
        # Check for failed login attempt (HTTP status code 401 or failure message)
        if "401" in line or "Invalid credentials" in line:
            # Extract IP address (assuming it's the first part of the log entry)
            ip = line.split()[0]
            suspicious_ips[ip] += 1

    # Filter IPs that exceed the threshold
    flagged_ips = {ip: count for ip, count in suspicious_ips.items() if count > threshold}
    return flagged_ips

# Set threshold for failed login attempts
# threshold of 10 that if any particular IP address has failed to log in more than 10 times, it will be flagged as suspicious.
# Here threshold value is 4 , the log file has few failed login attempts to display thet I have used the value < 10, if the value is 10 There will be no suspicious activity detected.
threshold = 4

# Detect suspicious activity
suspicious_activity = detect_suspicious_activity(log_lines, threshold)

# Display the results
if suspicious_activity:
    print("Suspicious Dectected Activity")
    print(f"\n{'IP Address':<20}{'Failed Login Attempts':<25}") 
    print("-" * 50)
    for ip, count in suspicious_activity.items():
        print(f"{ip:<20}{count:<25}")
else:
    print("No suspicious activity detected.")


Suspicious Dectected Activity

IP Address          Failed Login Attempts    
--------------------------------------------------
203.0.113.5         8                        
192.168.1.100       5                        


In [30]:
import csv

# Save results to CSV
def save_to_csv(ip_request_counts, most_accessed_endpoint, suspicious_activity, output_file="log_analysis_results.csv"):
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)

        # Write Requests per IP
        writer.writerow(["Requests per IP"])
        writer.writerow(["IP Address", "Request Count"])
        for ip, count in ip_request_counts:
            writer.writerow([ip, count])
        writer.writerow([])  # Blank row for separation

        # Write Most Accessed Endpoint
        writer.writerow(["Most Accessed Endpoint"])
        writer.writerow(["Endpoint", "Access Count"])
        if most_accessed_endpoint:
            endpoint, count = most_accessed_endpoint[0]
            writer.writerow([endpoint, count])
        writer.writerow([])  # Blank row for separation

        # Write Suspicious Activity
        writer.writerow(["Suspicious Activity"])
        writer.writerow(["IP Address", "Failed Login Count"])
        for ip, count in suspicious_activity.items():
            writer.writerow([ip, count])

# Display and Save Results
def display_and_save_results():
    # Requests per IP
    print("\nRequests per IP:")
    print(f"\n{'IP Address':<20}{'Request Count':<15}")
    print("-" * 35)
    for ip, count in ip_request_counts:
        print(f"{ip:<20}{count:<15}")

    # Most Accessed Endpoint
    print("\nMost Frequently Accessed Endpoint:")
    if most_accessed_endpoint:
        endpoint, count = most_accessed_endpoint[0]
        print(f"\n{'Endpoint':<20}{'Access Count':<15}")
        print("-" * 35)
        print(f"{endpoint:<20}{count:<15}")
    else:
        print("No endpoints found in the log.")

    # Suspicious Activity
    print("\nSuspicious Activity Detected:")
    if suspicious_activity:
        print(f"\n{'IP Address':<20}{'Failed Login Attempts':<25}")
        print("-" * 50)
        for ip, count in suspicious_activity.items():
            print(f"{ip:<20}{count:<25}")
    else:
        print("\nNo suspicious activity detected.")

    # Save to CSV
    save_to_csv(ip_request_counts, most_accessed_endpoint, suspicious_activity)
    print("\nResults have been saved to 'log_analysis_results.csv'.")

# Execute the pipeline
display_and_save_results()



Requests per IP:

IP Address          Request Count  
-----------------------------------
203.0.113.5         8              
198.51.100.23       8              
192.168.1.1         7              
10.0.0.2            6              
192.168.1.100       5              

Most Frequently Accessed Endpoint:

Endpoint            Access Count   
-----------------------------------
/login              13             

Suspicious Activity Detected:

IP Address          Failed Login Attempts    
--------------------------------------------------
203.0.113.5         8                        
192.168.1.100       5                        

Results have been saved to 'log_analysis_results.csv'.
